In [3]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

def entropy_image(image_path):
    # 读取PNG图片
    image = Image.open(image_path)
    original_array = np.array(image)

    # Calculate the histogram
    hist, _ = np.histogram(original_array.flatten(), bins=256, range=(0, 256))
    probabilities = hist / np.sum(hist)

    # Calculate the entropy
    entropy = -np.sum(probabilities * np.log2(probabilities + 1e-10))
    return entropy

encrypt_images = ['misc/encrypt_image_boat256.png','misc/encrypt_image_lena256.png','misc/encrypted_image_baboon.png']

for img in encrypt_images:
    print(img,"enntrop is :",entropy_image(img) )


misc/encrypt_image_boat256.png enntrop is : 7.997148587423335
misc/encrypt_image_lena256.png enntrop is : 7.997155261079634
misc/encrypted_image_baboon.png enntrop is : 7.997387415219332


In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

'''
分别计算图像通道相邻像素的水平、垂直和对角线的相关系数并返回
'''
def RGB_correlation(channel,N):
  #计算channel通道
  h,w=channel.shape
  #随机产生pixels个[0,w-1)范围内的整数序列
  row=np.random.randint(0,h-1,N)
  col=np.random.randint(0,w-1,N)
  #绘制相邻像素相关性图,统计x,y坐标
  x=[]
  h_y=[]
  v_y=[]
  d_y=[]

  for i in range(N):
    #选择当前一个像素
    x.append(channel[row[i]][col[i]])
    #水平相邻像素是它的右侧也就是同行下一列的像素
    h_y.append(channel[row[i]][col[i]+1])
    #垂直相邻像素是它的下方也就是同列下一行的像素
    v_y.append(channel[row[i]+1][col[i]])
    #对角线相邻像素是它的右下即下一行下一列的那个像素
    d_y.append(channel[row[i]+1][col[i]+1])
  
  
  #三个方向的合到一起
  x_init = x 
  x=x*3
  y=h_y+v_y+d_y

  #结果展示
  # plt.rcParams['font.sans-serif'] = ['SimHei']  # 中文乱码
  # plt.scatter(x,y)
  # plt.show()

  #计算E(x)，计算三个方向相关性时，x没有重新选择也可以更改
  ex=0
  for i in range(N):
    ex+=channel[row[i]][col[i]]
  ex=ex/N
  #计算D(x)
  dx=0
  for i in range(N):
    dx+=(channel[row[i]][col[i]]-ex)**2
  dx/=N

  #水平相邻像素h_y
  #计算E(y)
  h_ey=0
  for i in range(N):
    h_ey+=channel[row[i]][col[i]+1]
  h_ey/=N
  #计算D(y)
  h_dy=0
  for i in range(N):
    h_dy+=(channel[row[i]][col[i]+1]-h_ey)**2
  h_dy/=N
  #计算协方差
  h_cov=0
  for i in range(N):
    h_cov+=(channel[row[i]][col[i]]-ex)*(channel[row[i]][col[i]+1]-h_ey)
  h_cov/=N
  h_Rxy=h_cov/(np.sqrt(dx)*np.sqrt(h_dy))

  #垂直相邻像素v_y
  #计算E(y)
  v_ey=0
  for i in range(N):
    v_ey+=channel[row[i]+1][col[i]]
  v_ey/=N
  #计算D(y)
  v_dy=0
  for i in range(N):
    v_dy+=(channel[row[i]+1][col[i]]-v_ey)**2
  v_dy/=N
  #计算协方差
  v_cov=0
  for i in range(N):
    v_cov+=(channel[row[i]][col[i]]-ex)*(channel[row[i]+1][col[i]]-v_ey)
  v_cov/=N
  v_Rxy=v_cov/(np.sqrt(dx)*np.sqrt(v_dy))

  #对角线相邻像素d_y
  #计算E(y)
  d_ey=0
  for i in range(N):
    d_ey+=channel[row[i]+1][col[i]+1]
  d_ey/=N
  #计算D(y)
  d_dy=0
  for i in range(N):
    d_dy+=(channel[row[i]+1][col[i]+1]-d_ey)**2
  d_dy/=N
  #计算协方差
  d_cov=0
  for i in range(N):
    d_cov+=(channel[row[i]][col[i]]-ex)*(channel[row[i]+1][col[i]+1]-d_ey)
  d_cov/=N
  d_Rxy=d_cov/(np.sqrt(dx)*np.sqrt(d_dy))

  return h_Rxy,v_Rxy,d_Rxy,x,y,x_init,h_y, v_y, d_y


def pltImage(x,y,save_path,typCor):
  plt.cla()

  #结果展示
  plt.rcParams['font.sans-serif'] = ['SimHei']  # 中文乱码

  plt.scatter(x,y,s=1,c='blue')
  plt.ylim(0, 300)  # 设置纵坐标范围为0-300
  plt.xlim(0, 300)  # 设置纵坐标范围为0-300
  #plt.title('通道R')
  plt.xlabel('Pixel Value of location(x,y)')

  if typCor == 'h':
    plt.ylabel('Pixel Value of location(x+1,y)')
  elif typCor == 'v':
    plt.ylabel('Pixel Value of location(x,y+1)')
  else:
    plt.ylabel('Pixel Value of location(x+1,y+1)')

  plt.savefig(save_path)
  return Rxy[0:3]



'''
分别计算图像img的各通道相邻像素的相关系数，默认随机选取3000对相邻像素
'''
def correlation(img,N=4000):
  img=cv2.imread(img, cv2.IMREAD_GRAYSCALE)
  #print(img)
  h,w=img.shape
  Rxy=RGB_correlation(img,N)
  return Rxy


img='misc/encrypted_image_baboon.png'

Rxy=correlation(img)

#pltImage(Rxy[5],Rxy[6],'misc/cor/decrypted_image_baboon_h.png','h')
#pltImage(Rxy[5],Rxy[7],'misc/cor/decrypted_image_baboon_v.png','v')
#pltImage(Rxy[5],Rxy[8],'misc/cor/decrypted_image_baboon_d.png','d')



#输出结果保留四位有效数字
print("******该图像的各方向的相关系数为*****")
print('\tHorizontal\tVertical\tDiagonal')
print('  \t{:.4f}    {:.4f}    {:.4f}'.format(Rxy[0],Rxy[1],Rxy[2]))


******该图像的各方向的相关系数为*****
	Horizontal	Vertical	Diagonal
  	-0.0254    0.0221    -0.0083
